In [1]:
# -*- coding: utf-8 -*-

# 说明
此脚本作用是在将EXCEL(.xlsx/.csv/.txt)导入数仓（impala/MySQL）前的数据检测

包括以下几点：

<img style="float: left;" src="https://www.yunqishi.net/uploads/allimg/2020/0715/acvgdhg2yhe.jpg" width="3%"> 检查并处理是否有NULL值或者空值

<img style="float: left;" src="https://www.yunqishi.net/uploads/allimg/2020/0715/acvgdhg2yhe.jpg" width="3%"> 单元格中是否有换行符

<img style="float: left;" src="https://www.yunqishi.net/uploads/allimg/2020/0715/acvgdhg2yhe.jpg" width="3%"> 单元格中是否有","

<img style="float: left;" src="https://www.yunqishi.net/uploads/allimg/2020/0715/acvgdhg2yhe.jpg" width="3%"> 检查每列数据类型

<img style="float: left;" src="https://www.yunqishi.net/uploads/allimg/2020/0715/acvgdhg2yhe.jpg" width="3%"> DataFrame最符合的建表类型


如果需要直接将Dataframe上传到Impala,需要安装sqlalchemy，使用如下:

from sqlalchemy import create_engine

from sqlalchemy.dialects.sqlite import INTEGER, VARCHAR, FLOAT

In [2]:
import re
import time
import sys
import json
import numpy as np
import pandas as pd
import xlwt,xlrd
import xlsxwriter
from datetime import datetime

In [3]:
file = "财务滚动预估/source_data/2020-12.xlsx"
sheetName = "ABM汇总文件"
df = pd.read_excel(file, index_col = None, header = 1, sheet_name = sheetName)

In [4]:
df

,品牌名称,产品名称,是否为赠品,商品编码,等级,RRP,2121-01-21 00:00:00,2121-02-21 00:00:00,2121-03-21 00:00:00,2121-04-21 00:00:00,...,Apr-22 GMV,May-22 GMV,Jun-22 GMV,Jul-22 GMV,Aug-22 GMV,Sep-22 GMV,Oct-22 GMV,Nov-22 GMV,Dec-22 GMV,2022 Total GMV
0,Alissi Bronte,控油抗痘安瓶,0,8435112200222,AC,696.0,557.0,658.0,873.0,712.0,...,1.113784e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.983869e+05
1,Alissi Bronte,ALISSIBRONTE瓜拉纳眼部紧致精华霜30ML,0,8435112201076,AB,596.0,704.0,553.0,1085.0,1173.0,...,2.392467e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.080882e+06
2,Alissi Bronte,ALISSIBRONTE黄金玫瑰活肤精华水200ML,0,8435112210573,AB,456.0,10392.0,2974.0,13028.0,13460.0,...,4.095706e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.494462e+07
3,Alissi Bronte,ALISSIBRONTE黄金泡沫洁面乳200ML,0,8435112210993,AB,356.0,9082.0,2533.0,11673.0,11182.0,...,2.388382e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.147263e+06
4,Alissi Bronte,ALISSIBRONTE精准祛痘套组,0,8435112228684,AB,496.0,726.0,857.0,1137.0,928.0,...,1.033735e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.481930e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,ch22,干湿两用棉柔巾（Go）,0,6972569300065,AC,89.0,1036.0,518.0,2246.0,1036.0,...,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
640,ch22,本色抽纸（Home）,0,6973346970082,AB,59.0,9720.0,4560.0,12480.0,9360.0,...,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
641,ch22,溶水有芯卷纸（Home）,0,6973346970105,AC,69.0,8640.0,3840.0,9600.0,5760.0,...,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
642,ch22,花瓣纸便携装1提,0,6972569300249,AC,69.0,8316.0,3696.0,9240.0,5544.0,...,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00


## 获取行数、列数、索引（Acquire Row, Column, index numbers）

In [5]:
row_num = df.shape[0]
col_num = df.shape[1]
print("row numbers:{},col numbers:{}".format(df.shape[0], df.shape[1]))

row numbers:644,col numbers:58


In [6]:
print("row index range:0~{}".format(df.shape[0]-1))
print("col index range:0~{}".format(df.shape[1]-1))

row index range:0~643
col index range:0~57


#### <img style="float: left;" src="https://webms1.xhd.cn//u/cms/wuhan/201809/041538140r3p.jpg" width="3%">常见Dataframe取值方法

In [7]:
df.iloc[0:1,0:1]

,品牌名称
0,Alissi Bronte


In [8]:
# 取（1,3）（1,4）（1,5）（2,5）
df.iloc[[0, 1],[2,4]]

,是否为赠品,等级
0,0,AC
1,0,AB


## <img style="float: left;" src="https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1616527064,266674329&fm=26&gp=0.jpg" width="2%">检查是否有NULL值或者空值（Check Null/Empty Values）

In [9]:
for index, row  in df.iterrows():
    for i in range(df.shape[1]):
        if(pd.isnull(row.iloc[i])):
            print('There is a Null/Empty value in ({},{})'.format(index, i))
        i += 1

There is a Null/Empty value in (0,49)
There is a Null/Empty value in (0,50)
There is a Null/Empty value in (0,51)
There is a Null/Empty value in (0,52)
There is a Null/Empty value in (0,53)
There is a Null/Empty value in (0,54)
There is a Null/Empty value in (0,55)
There is a Null/Empty value in (0,56)
There is a Null/Empty value in (1,49)
There is a Null/Empty value in (1,50)
There is a Null/Empty value in (1,51)
There is a Null/Empty value in (1,52)
There is a Null/Empty value in (1,53)
There is a Null/Empty value in (1,54)
There is a Null/Empty value in (1,55)
There is a Null/Empty value in (1,56)
There is a Null/Empty value in (2,49)
There is a Null/Empty value in (2,50)
There is a Null/Empty value in (2,51)
There is a Null/Empty value in (2,52)
There is a Null/Empty value in (2,53)
There is a Null/Empty value in (2,54)
There is a Null/Empty value in (2,55)
There is a Null/Empty value in (2,56)
There is a Null/Empty value in (3,49)
There is a Null/Empty value in (3,50)
There is a N

There is a Null/Empty value in (33,55)
There is a Null/Empty value in (33,56)
There is a Null/Empty value in (34,49)
There is a Null/Empty value in (34,50)
There is a Null/Empty value in (34,51)
There is a Null/Empty value in (34,52)
There is a Null/Empty value in (34,53)
There is a Null/Empty value in (34,54)
There is a Null/Empty value in (34,55)
There is a Null/Empty value in (34,56)
There is a Null/Empty value in (35,49)
There is a Null/Empty value in (35,50)
There is a Null/Empty value in (35,51)
There is a Null/Empty value in (35,52)
There is a Null/Empty value in (35,53)
There is a Null/Empty value in (35,54)
There is a Null/Empty value in (35,55)
There is a Null/Empty value in (35,56)
There is a Null/Empty value in (36,49)
There is a Null/Empty value in (36,50)
There is a Null/Empty value in (36,51)
There is a Null/Empty value in (36,52)
There is a Null/Empty value in (36,53)
There is a Null/Empty value in (36,54)
There is a Null/Empty value in (36,55)
There is a Null/Empty val

There is a Null/Empty value in (76,53)
There is a Null/Empty value in (76,54)
There is a Null/Empty value in (76,55)
There is a Null/Empty value in (76,56)
There is a Null/Empty value in (77,49)
There is a Null/Empty value in (77,50)
There is a Null/Empty value in (77,51)
There is a Null/Empty value in (77,52)
There is a Null/Empty value in (77,53)
There is a Null/Empty value in (77,54)
There is a Null/Empty value in (77,55)
There is a Null/Empty value in (77,56)
There is a Null/Empty value in (78,49)
There is a Null/Empty value in (78,50)
There is a Null/Empty value in (78,51)
There is a Null/Empty value in (78,52)
There is a Null/Empty value in (78,53)
There is a Null/Empty value in (78,54)
There is a Null/Empty value in (78,55)
There is a Null/Empty value in (78,56)
There is a Null/Empty value in (79,49)
There is a Null/Empty value in (79,50)
There is a Null/Empty value in (79,51)
There is a Null/Empty value in (79,52)
There is a Null/Empty value in (79,53)
There is a Null/Empty val

There is a Null/Empty value in (116,49)
There is a Null/Empty value in (116,50)
There is a Null/Empty value in (116,51)
There is a Null/Empty value in (116,52)
There is a Null/Empty value in (116,53)
There is a Null/Empty value in (116,54)
There is a Null/Empty value in (116,55)
There is a Null/Empty value in (116,56)
There is a Null/Empty value in (117,49)
There is a Null/Empty value in (117,50)
There is a Null/Empty value in (117,51)
There is a Null/Empty value in (117,52)
There is a Null/Empty value in (117,53)
There is a Null/Empty value in (117,54)
There is a Null/Empty value in (117,55)
There is a Null/Empty value in (117,56)
There is a Null/Empty value in (118,49)
There is a Null/Empty value in (118,50)
There is a Null/Empty value in (118,51)
There is a Null/Empty value in (118,52)
There is a Null/Empty value in (118,53)
There is a Null/Empty value in (118,54)
There is a Null/Empty value in (118,55)
There is a Null/Empty value in (118,56)
There is a Null/Empty value in (119,49)


There is a Null/Empty value in (156,5)
There is a Null/Empty value in (156,6)
There is a Null/Empty value in (156,7)
There is a Null/Empty value in (156,8)
There is a Null/Empty value in (156,9)
There is a Null/Empty value in (156,10)
There is a Null/Empty value in (156,11)
There is a Null/Empty value in (156,12)
There is a Null/Empty value in (156,13)
There is a Null/Empty value in (156,14)
There is a Null/Empty value in (156,15)
There is a Null/Empty value in (156,16)
There is a Null/Empty value in (156,17)
There is a Null/Empty value in (156,18)
There is a Null/Empty value in (156,32)
There is a Null/Empty value in (156,33)
There is a Null/Empty value in (156,34)
There is a Null/Empty value in (156,35)
There is a Null/Empty value in (156,36)
There is a Null/Empty value in (156,37)
There is a Null/Empty value in (156,38)
There is a Null/Empty value in (156,39)
There is a Null/Empty value in (156,40)
There is a Null/Empty value in (156,41)
There is a Null/Empty value in (156,42)
There

...

There is a Null/Empty value in (348,54)
There is a Null/Empty value in (348,55)
There is a Null/Empty value in (348,56)
There is a Null/Empty value in (349,49)
There is a Null/Empty value in (349,50)
There is a Null/Empty value in (349,51)
There is a Null/Empty value in (349,52)
There is a Null/Empty value in (349,53)
There is a Null/Empty value in (349,54)
There is a Null/Empty value in (349,55)
There is a Null/Empty value in (349,56)
There is a Null/Empty value in (350,49)
There is a Null/Empty value in (350,50)
There is a Null/Empty value in (350,51)
There is a Null/Empty value in (350,52)
There is a Null/Empty value in (350,53)
There is a Null/Empty value in (350,54)
There is a Null/Empty value in (350,55)
There is a Null/Empty value in (350,56)
There is a Null/Empty value in (351,49)
There is a Null/Empty value in (351,50)
There is a Null/Empty value in (351,51)
There is a Null/Empty value in (351,52)
There is a Null/Empty value in (351,53)
There is a Null/Empty value in (351,54)


There is a Null/Empty value in (411,49)
There is a Null/Empty value in (411,50)
There is a Null/Empty value in (411,51)
There is a Null/Empty value in (411,52)
There is a Null/Empty value in (411,53)
There is a Null/Empty value in (411,54)
There is a Null/Empty value in (411,55)
There is a Null/Empty value in (411,56)
There is a Null/Empty value in (412,49)
There is a Null/Empty value in (412,50)
There is a Null/Empty value in (412,51)
There is a Null/Empty value in (412,52)
There is a Null/Empty value in (412,53)
There is a Null/Empty value in (412,54)
There is a Null/Empty value in (412,55)
There is a Null/Empty value in (412,56)
There is a Null/Empty value in (413,49)
There is a Null/Empty value in (413,50)
There is a Null/Empty value in (413,51)
There is a Null/Empty value in (413,52)
There is a Null/Empty value in (413,53)
There is a Null/Empty value in (413,54)
There is a Null/Empty value in (413,55)
There is a Null/Empty value in (413,56)
There is a Null/Empty value in (414,49)


There is a Null/Empty value in (475,49)
There is a Null/Empty value in (475,50)
There is a Null/Empty value in (475,51)
There is a Null/Empty value in (475,52)
There is a Null/Empty value in (475,53)
There is a Null/Empty value in (475,54)
There is a Null/Empty value in (475,55)
There is a Null/Empty value in (475,56)
There is a Null/Empty value in (476,49)
There is a Null/Empty value in (476,50)
There is a Null/Empty value in (476,51)
There is a Null/Empty value in (476,52)
There is a Null/Empty value in (476,53)
There is a Null/Empty value in (476,54)
There is a Null/Empty value in (476,55)
There is a Null/Empty value in (476,56)
There is a Null/Empty value in (477,49)
There is a Null/Empty value in (477,50)
There is a Null/Empty value in (477,51)
There is a Null/Empty value in (477,52)
There is a Null/Empty value in (477,53)
There is a Null/Empty value in (477,54)
There is a Null/Empty value in (477,55)
There is a Null/Empty value in (477,56)
There is a Null/Empty value in (478,49)


There is a Null/Empty value in (533,49)
There is a Null/Empty value in (533,50)
There is a Null/Empty value in (533,51)
There is a Null/Empty value in (533,52)
There is a Null/Empty value in (533,53)
There is a Null/Empty value in (533,54)
There is a Null/Empty value in (533,55)
There is a Null/Empty value in (533,56)
There is a Null/Empty value in (534,49)
There is a Null/Empty value in (534,50)
There is a Null/Empty value in (534,51)
There is a Null/Empty value in (534,52)
There is a Null/Empty value in (534,53)
There is a Null/Empty value in (534,54)
There is a Null/Empty value in (534,55)
There is a Null/Empty value in (534,56)
There is a Null/Empty value in (535,49)
There is a Null/Empty value in (535,50)
There is a Null/Empty value in (535,51)
There is a Null/Empty value in (535,52)
There is a Null/Empty value in (535,53)
There is a Null/Empty value in (535,54)
There is a Null/Empty value in (535,55)
There is a Null/Empty value in (535,56)
There is a Null/Empty value in (536,49)


There is a Null/Empty value in (589,50)
There is a Null/Empty value in (589,51)
There is a Null/Empty value in (589,52)
There is a Null/Empty value in (589,53)
There is a Null/Empty value in (589,54)
There is a Null/Empty value in (589,55)
There is a Null/Empty value in (589,56)
There is a Null/Empty value in (590,49)
There is a Null/Empty value in (590,50)
There is a Null/Empty value in (590,51)
There is a Null/Empty value in (590,52)
There is a Null/Empty value in (590,53)
There is a Null/Empty value in (590,54)
There is a Null/Empty value in (590,55)
There is a Null/Empty value in (590,56)
There is a Null/Empty value in (591,49)
There is a Null/Empty value in (591,50)
There is a Null/Empty value in (591,51)
There is a Null/Empty value in (591,52)
There is a Null/Empty value in (591,53)
There is a Null/Empty value in (591,54)
There is a Null/Empty value in (591,55)
There is a Null/Empty value in (591,56)
There is a Null/Empty value in (592,49)
There is a Null/Empty value in (592,50)


## <img style="float: left;" src="https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1616527064,266674329&fm=26&gp=0.jpg" width="2%">处理NULL值或者空值（Check Null/Empty Values）

In [10]:
new_df = df.dropna()
if new_df.shape[0]/df.shape[0]  > 0.95:
    print("There are {} rows are deleted due to Null/Empty Values".format(df.shape[0] - new_df.shape[0]))
else:
    print("Too many Null/Empty Values, Null/Empty Values should be replaced instead of been dropped")
    df = df.fillna(0)
    print("Null/Empty Values have been replced with 0")

Too many Null/Empty Values, Null/Empty Values should be replaced instead of been dropped
Null/Empty Values have been replced with 0


In [11]:
df

,品牌名称,产品名称,是否为赠品,商品编码,等级,RRP,2121-01-21 00:00:00,2121-02-21 00:00:00,2121-03-21 00:00:00,2121-04-21 00:00:00,...,Apr-22 GMV,May-22 GMV,Jun-22 GMV,Jul-22 GMV,Aug-22 GMV,Sep-22 GMV,Oct-22 GMV,Nov-22 GMV,Dec-22 GMV,2022 Total GMV
0,Alissi Bronte,控油抗痘安瓶,0,8435112200222,AC,696.0,557.0,658.0,873.0,712.0,...,1.113784e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.983869e+05
1,Alissi Bronte,ALISSIBRONTE瓜拉纳眼部紧致精华霜30ML,0,8435112201076,AB,596.0,704.0,553.0,1085.0,1173.0,...,2.392467e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.080882e+06
2,Alissi Bronte,ALISSIBRONTE黄金玫瑰活肤精华水200ML,0,8435112210573,AB,456.0,10392.0,2974.0,13028.0,13460.0,...,4.095706e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.494462e+07
3,Alissi Bronte,ALISSIBRONTE黄金泡沫洁面乳200ML,0,8435112210993,AB,356.0,9082.0,2533.0,11673.0,11182.0,...,2.388382e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.147263e+06
4,Alissi Bronte,ALISSIBRONTE精准祛痘套组,0,8435112228684,AB,496.0,726.0,857.0,1137.0,928.0,...,1.033735e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.481930e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,ch22,干湿两用棉柔巾（Go）,0,6972569300065,AC,89.0,1036.0,518.0,2246.0,1036.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
640,ch22,本色抽纸（Home）,0,6973346970082,AB,59.0,9720.0,4560.0,12480.0,9360.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
641,ch22,溶水有芯卷纸（Home）,0,6973346970105,AC,69.0,8640.0,3840.0,9600.0,5760.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
642,ch22,花瓣纸便携装1提,0,6972569300249,AC,69.0,8316.0,3696.0,9240.0,5544.0,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00


## <img style="float: left;" src="https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1616527064,266674329&fm=26&gp=0.jpg" width="2%">替换单元格中换行符（Replace \n, \r, or \r\n, ）

In [12]:
df = df.replace('\r\n', '', regex=True).replace('\n', '', regex=True).replace('\r', '', regex=True)

## <img style="float: left;" src="https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1616527064,266674329&fm=26&gp=0.jpg" width="2%">替换单元格中','（Replace ',' ）

#### <img style="float: left;" src="https://webms1.xhd.cn//u/cms/wuhan/201809/041538140r3p.jpg" width="3%">如果不替换，在导入impala时，将会使用","作为分隔符，会导致文本单元格中的字符串延伸至其他单元格

In [13]:
for i in range(df.shape[0] - 1):
    for j in range(df.shape[1] - 1):
        if(isinstance(df.iloc[i,j],str)):
            df.iloc[i,j] = df.iloc[i,j].replace(",","_")
            df = df.apply(lambda x: x.rstrip() if type(x) == str else x)
        j =+ 1
    i =+ 1

## <img style="float: left;" src="https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1616527064,266674329&fm=26&gp=0.jpg" width="2%">Save to local logs

In [14]:
now = time.strftime('%Y-%m-%d-%H-%M-%S',time.localtime(int(round(time.time()*1000))/1000))
df.to_csv("log/file_output-{}.txt".format(now), encoding="utf-8", index=False)

## <img style="float: left;" src="https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1616527064,266674329&fm=26&gp=0.jpg" width="2%">DataFrame最符合的建表类型（Most suitable table scheme ）

In [15]:
from sqlalchemy import create_engine
from sqlalchemy.dialects.sqlite import INTEGER, VARCHAR, FLOAT

#### <img style="float: left;" src="https://webms1.xhd.cn//u/cms/wuhan/201809/041538140r3p.jpg" width="3%">查看基础属性

In [16]:
df.dtypes

品牌名称                    object
产品名称                    object
是否为赠品                    int64
商品编码                    object
等级                      object
RRP                    float64
2121-01-21 00:00:00    float64
2121-02-21 00:00:00    float64
2121-03-21 00:00:00    float64
2121-04-21 00:00:00    float64
2021-05-21 00:00:00    float64
2021-06-21 00:00:00    float64
2021-07-21 00:00:00    float64
2021-08-21 00:00:00    float64
2021-09-21 00:00:00    float64
2021-10-21 00:00:00    float64
2021-11-21 00:00:00    float64
2021-12-21 00:00:00    float64
2021 Total Qty         float64
Jan-21 GMV             float64
Feb-21 GMV             float64
Mar-21 GMV             float64
Apr-21 GMV             float64
May-21 GMV             float64
Jun-21 GMV             float64
Jul-21 GMV             float64
Aug-21 GMV             float64
Sep-21 GMV             float64
Oct-21 GMV             float64
Nov-21 GMV             float64
Dec-21 GMV             float64
2021 Total GMV         float64
2021-01-

In [17]:
def get_dict_type(df):
    type_dict = {}
    for col in df.columns:
        tp = df[col].dtype
        if 'object' in str(tp):
            type_dict[col] = VARCHAR()
            df[col] = df[col].apply(lambda x: str(x))
        elif 'int' in str(tp):
            type_dict[col] = INTEGER()
        elif 'float' in str(tp):
            type_dict[col] = FLOAT()
        else:
            type_dict[col] = VARCHAR()
    return type_dict

In [18]:
type_dict =  get_dict_type(df)

In [19]:
type_dict

{'品牌名称': VARCHAR(),
 '产品名称': VARCHAR(),
 '是否为赠品': INTEGER(),
 '商品编码': VARCHAR(),
 '等级': VARCHAR(),
 'RRP': FLOAT(),
 datetime.datetime(2121, 1, 21, 0, 0): FLOAT(),
 datetime.datetime(2121, 2, 21, 0, 0): FLOAT(),
 datetime.datetime(2121, 3, 21, 0, 0): FLOAT(),
 datetime.datetime(2121, 4, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 5, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 6, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 7, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 8, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 9, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 10, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 11, 21, 0, 0): FLOAT(),
 datetime.datetime(2021, 12, 21, 0, 0): FLOAT(),
 '2021 Total Qty': FLOAT(),
 'Jan-21 GMV': FLOAT(),
 'Feb-21 GMV': FLOAT(),
 'Mar-21 GMV': FLOAT(),
 'Apr-21 GMV': FLOAT(),
 'May-21 GMV': FLOAT(),
 'Jun-21 GMV': FLOAT(),
 'Jul-21 GMV': FLOAT(),
 'Aug-21 GMV': FLOAT(),
 'Sep-21 GMV': FLOAT(),
 'Oct-21 GMV': FLOAT(),
 'Nov-21 GMV': FLOAT(),
 'Dec-21 GMV'

In [20]:
# engine = create_engine('impala://{}:{}/{}'.format('impala.acg.team', '21050', 'access_aus_bi'))

# # if_exists 如果表存在就做什么操作，可以等于 {'fail', 'replace', 'append'}
# df_type_dict = get_dict_type(df)
# df_result.to_sql('impala_table', con=engine, if_exists='replace', index=False, dtype=df_type_dict)
# #